In [11]:
# All imports go here
import polars as pl

In [12]:
endpoint = "https://github.com/DataTalksClub/nyc-tlc-data/releases/download/yellow/yellow_tripdata_2019-01.csv.gz"

In [13]:
def yellow_taxi_polars_schema() -> dict:
  return {
    "VendorID": pl.Int32,
    "tpep_pickup_datetime": pl.Datetime,
    "tpep_dropoff_datetime": pl.Datetime,
    "passenger_count": pl.Int8,
    "trip_distance": pl.Float64,
    "PULocationID": pl.Int32,
    "DOLocationID": pl.Int32,
    "RatecodeID": pl.Int8,
    "store_and_fwd_flag": pl.String,
    "payment_type": pl.Int8,
    "fare_amount": pl.Float64,
    "extra": pl.Float64,
    "mta_tax": pl.Float64,
    "improvement_surcharge": pl.Float64,
    "tip_amount": pl.Float64,
    "tolls_amount": pl.Float64,
    "total_amount": pl.Float64,
    "congestion_surcharge": pl.Float64,
  }

In [17]:
df = pl.read_csv(endpoint, dtypes=yellow_taxi_polars_schema(), n_rows=100)

/var/folders/5d/s1g43qys4psf6zbcy2y3tbsh0000gp/T/ipykernel_8760/399295492.py:1: DeprecationWarning: The argument `dtypes` for `read_csv` is deprecated. It has been renamed to `schema_overrides`.
  df = pl.read_csv(endpoint, dtypes=yellow_taxi_polars_schema(), n_rows=100)


In [18]:
df

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
i32,datetime[μs],datetime[μs],i8,f64,i8,str,i32,i32,i8,f64,f64,f64,f64,f64,f64,f64,f64
1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,1,"""N""",151,239,1,7.0,0.5,0.5,1.65,0.0,0.3,9.95,null
1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.6,1,"""N""",239,246,1,14.0,0.5,0.5,1.0,0.0,0.3,16.3,null
2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0.0,1,"""N""",236,236,1,4.5,0.5,0.5,0.0,0.0,0.3,5.8,null
2,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0.0,1,"""N""",193,193,2,3.5,0.5,0.5,0.0,0.0,0.3,7.55,null
2,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0.0,2,"""N""",193,193,2,52.0,0.0,0.5,0.0,0.0,0.3,55.55,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2,2019-01-01 00:02:06,2019-01-01 00:10:19,2,2.57,1,"""N""",45,97,1,9.5,0.5,0.5,2.16,0.0,0.3,12.96,null
2,2019-01-01 00:24:57,2019-01-01 00:35:42,2,2.44,1,"""N""",144,256,2,10.5,0.5,0.5,0.0,0.0,0.3,11.8,null
2,2019-01-01 00:46:09,2019-01-01 01:06:46,1,4.18,1,"""N""",79,263,1,16.0,0.5,0.5,3.46,0.0,0.3,20.76,null


In [16]:
df.write_database(
  table_name="yellow_taxi_data",
  connection="postgres://{user}:{password}@{host}:{post}/{db_name}",
  engine="adbc",
  if_table_exists="replace",
)

'\ndf.write_database(\n  table_name="yellow_taxi_data",\n  connection="postgres://{user}:{password}@{host}:{post}/{db_name}",\n  engine="adbc",\n  if_table_exists="replace",\n)\n'